In [1]:
import os 
import numpy as np
import random
import math
from sklearn.metrics import average_precision_score
import pandas as pd
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
#從grid_search_weight中找尋不同的file 
path = 'D:/ChilliHsu/Data/grid_search_weight/Our/'
all_files = os.listdir(path)
files = [file for file in all_files if 'ALL' in file and '.npz' in file]
files
files = ['ALL_2_0.npz']

In [4]:
def square_sum(pars):
    all_squ_sum = []
    for par in pars:
        all_squ_sum.append(np.sum(np.multiply(par,par)))
    return all_squ_sum 
def shape_sum(pars):
    all_shape = []
    for par in pars:
        all_shape.append(par.shape)
    return all_shape
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(z):
    return 1 / (1 + np.exp(-z))  

In [5]:
user_following = np.load('../Data/npy/user_following_1489.npy')
all_3374 = np.load('../Data/npy/all_2939D_img0.5.npy')
user_category = np.load('../Data/npy/user_category_1489.npy')
YouTuber_category = np.load('../Data/npy/YouTuber_category_0.7.npy')
active_users = np.load('../Data/npy/active_userID_1489.npy')

In [6]:
def get_test_idx(num):
    
    is_used = []
    for i in range(num+1):
        random.seed(5)
        all_test_idx = [i for i in range(len(user_following))]
        left_idx = [i for i in all_test_idx if i not in is_used]
        test_idx = sorted(random.sample(left_idx,150))
        is_used = is_used+test_idx
    return test_idx

In [7]:
user_category_norm = np.zeros(user_category.shape)
for i in range(len(user_category)):
    user_category_norm[i] = user_category[i]/np.max(user_category[i])
print('user_category after normalized by max...')
print('user_category_norm shape ',user_category_norm.shape)
print('user cateogory norm',user_category_norm)
following_true = [0]*len(user_following)
for i in range(len(user_following)):
    each_user = []
    for j in range(len(user_following[i])):
        if user_following[i][j] == 1:
            each_user.append(j)
    following_true[i] = each_user
#print(following_true)
#最少跟最多的following 
minlen = 10000
maxlen = 0
num_of_follower = []
for i in range(len(following_true)):
    if len(following_true[i]) < minlen:
        minlen = len(following_true[i])
    if len(following_true[i]) > maxlen:
        maxlen = len(following_true[i])
    num_of_follower.append(len(following_true[i]))
print('Min number of followings ',minlen)
print('Max number of followings ',maxlen)
test_amount = 150
yt_test_amount = 18
user_idx = [i for i in range(len(user_following))]
#user_idx = user_idx_over10
#test_idx is the number of user for testing
random.seed(5)
#test_idx = sorted(random.sample(user_idx,test_amount))
test_idx = get_test_idx(0)



print(test_idx)
# Training  and Testing --New
train_t = [0]*(len(user_following))
train_f = [0]*(len(user_following))
# Testing 
test_t = [0]*test_amount
test_f = [0]*test_amount
test_pos = -1

for i in range(len(user_following)):
    t_for_train = []
    f_for_train = []
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(88):
            if user_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
        train_t[i] = t_for_train
        train_f[i] = f_for_train
        
    else: #if in test id, choose 2 true and other 
        test_pos += 1
        temp_t = []
        temp_f = []
        for j in range(88):
            if user_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        #print(len(temp_t),math.ceil(0.5*len(temp_t)))
        t_for_test = random.sample(temp_t,math.ceil(0.5*len(temp_t)))
        f_for_test  = random.sample(temp_f,yt_test_amount-len(t_for_test))
        
        test_t[test_pos] = t_for_test
        test_f[test_pos] = f_for_test
        
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        #print(len(t_for_train ))
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t[i] = t_for_train
        train_f[i] = f_for_train
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/test_amount
print(avg)

user_category after normalized by max...
user_category_norm shape  (1489, 17)
user cateogory norm [[0.         1.         0.         ... 0.05714286 0.         0.        ]
 [0.24390244 0.         0.02439024 ... 0.         0.         0.09756098]
 [0.04210526 0.04210526 0.05263158 ... 0.02105263 0.         0.02105263]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.01408451 0.01408451 0.04225352 ... 0.02816901 0.         0.02816901]
 [0.03703704 0.22222222 0.14814815 ... 0.         0.         0.        ]]
Min number of followings  5
Max number of followings  34
[3, 6, 10, 18, 26, 37, 44, 46, 59, 65, 67, 75, 95, 99, 106, 114, 116, 133, 135, 147, 160, 165, 186, 188, 208, 221, 231, 243, 259, 270, 284, 298, 303, 304, 321, 326, 330, 339, 340, 360, 363, 370, 372, 378, 402, 403, 407, 419, 426, 428, 441, 443, 486, 503, 504, 510, 512, 513, 523, 524, 540, 564, 572, 592, 605, 611, 617, 626, 627, 633, 634, 639, 642, 646, 648, 679, 692, 696, 697, 704, 705, 726, 727, 7

In [8]:
index = 0 
for train_idx in range(len(train_t)):
    if train_idx in test_idx:
        #print(len(train_t[train_idx])+len(train_f[train_idx])+len(test_t[index])+len(test_f[index]))
        #print(train_f[train_idx])
        #print(test_f[index])
        all_idx = train_t[train_idx]+train_f[train_idx]+test_t[index]+test_f[index]
        print(len(sorted(list(set(all_idx)))))
        index+=1
    else:
        all_idx = train_t[train_idx]+train_f[train_idx]
        print(len(sorted(list(set(all_idx)))))

88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
8

In [9]:
def getScoreMatrix(RS):
    #取出test的資料
    print('RS shape',RS.shape)
    testRS = np.zeros((test_amount,yt_test_amount)) #shape 150*18
    target = np.zeros((test_amount,yt_test_amount))
    #test_t 是true的
    #test_f 是false的

    for z in range(test_amount):
        user_id = test_idx[z]
        #positive target YouTuber list
        youtube_t = test_t[z] 
        #not target YouTuber list
        youtube_f = test_f[z]

        #前兩個放target的RS
        for i in range(len(youtube_t)):
            testRS[z][i] = RS[z][youtube_t[i]]
            target[z][i] = user_following[user_id][youtube_t[i]]
        for i in range(len(youtube_f)):
            testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]
            target[z][i+len(youtube_t)] = user_following[user_id][youtube_f[i]]
    sumtarget = 0
    for i in range(len(target)):
        #print(np.sum(target[i]))
        sumtarget += np.sum(target[i])
    print('num of positive data in testing:',sumtarget)
    print('total testing data:',test_amount*yt_test_amount)
    return target, testRS,sumtarget

def topN(sortlist,n):
    topList = []
    for i in range(n):
        topList.append(sortlist.index(max(sortlist)))
        #print(max(sortlist))
        #print(sortlist.index(max(sortlist)))
        sortlist[sortlist.index(max(sortlist))] = -1000000000
    return topList

def F1_score(prec,rec):
    f1 = (2*prec*rec)/(prec+rec)
    return f1

In [10]:
def getTOP1(target,testRS,sumtarget):
    print('top1')
    correct = 0
    for i in range(len(testRS)):
        top_0 = topN(list(testRS[i]),1) #取一個
        
        #print(top_0)
        if top_0[0] < int(np.sum(target[i])):
            correct += 1
    top1_prec = correct/len(testRS)
    top1_recall = correct/(sumtarget)
    print('prec ',top1_prec,'recall ',top1_recall)
    #f1 score
    print('F1_score:',F1_score(top1_prec,top1_recall))
    return top1_prec,top1_recall,F1_score(top1_prec,top1_recall)
def getTOP3(target,testRS,sumtarget):
    print('top3')
    correct = 0
    for i in range(len(testRS)):
        top_3 = topN(list(testRS[i]),3) #取一個
        
        #print(top_3)
        for j in range(len(top_3)):
            if top_3[j] < int(np.sum(target[i])):
                correct += 1
    top3_prec = correct/(len(testRS)*3)
    top3_recall = correct/(sumtarget)
    print('prec ',top3_prec,'recall ',top3_recall)
    #f1 score
    print('F1_score:',F1_score(top3_prec,top3_recall))
    return top3_prec,top3_recall,F1_score(top3_prec,top3_recall)
def getTOP5(target,testRS,sumtarget):
    print('top5')
    correct = 0
    for i in range(len(testRS)):
        top_5 = topN(list(testRS[i]),5) #取一個
       
        #print(top_5)
        for j in range(len(top_5)):
            if top_5[j] < int(np.sum(target[i])):
                correct += 1
    top5_prec = correct/(len(testRS)*5)
    top5_recall = correct/(sumtarget)
    print('prec ',top5_prec,'recall ',top5_recall)
    #f1 score
    print('F1_score:',F1_score(top5_prec,top5_recall))
    return top5_prec,top5_recall,F1_score(top5_prec,top5_recall)

In [11]:
"""
NDCG
https://daiwk.github.io/posts/nlp-ndcg.html
"""
# pre_list
"""
test_amount = 150
yt_test_amount = 18
"""
def NDCG(target,testRS): #target是真正的喜好
    all_sort = []
    num_ndcg = 10
    pre_matrix = np.zeros(shape=(test_amount,yt_test_amount)) #(150,18)
    for i in range(test_amount): #user amount = 150
        top_n = topN(list(testRS[i]),num_ndcg) #取10個
        #print(top_n)
        all_sort.append(topN(list(testRS[i]),len(testRS[i])))
        #print('all_sort',topN(list(testRS[i]),len(testRS[i])))
        for j in range(len(top_n)):
            pre_matrix[i][top_n[j]] = 1

    #Ideal DCG，理想状况下的DCG。也就是说，相关性完全由高到低排序时算出的DCG：
    def IDCG(ideal_list): #ideal_list example = [1,1,1,1,1,0,0,....]
        idcg=0
        #print('ideal',ideal_list)
        for i in range(len(ideal_list)):
            #print((2**true_list[i]-1),math.log2(i+2))
            idcg+= (2**ideal_list[i]-1)/ math.log2(i+2)
        #print('idcg',idcg)
        return idcg
    def DCG(prec_list): #找出前n名的[1,1,1,0,...]
        dcg=0
        #print('prec',prec_list)
        for i in range(len(prec_list)):
            dcg+= (2**prec_list[i]-1)/ math.log2(i+2)
        #print('dcg',dcg)
        return dcg
    total_ndcg = 0
    
    for m in range(test_amount): # the number of testing users
        idcg = IDCG(target[m][:num_ndcg])
        pre_list = []
        least_pre_list = []
        for s in all_sort[m][:num_ndcg]:
            #print(m,s,target[m][s])
            pre_list.append(target[m][s]) #把prec_list 的 score加進去
        for s in all_sort[m][num_ndcg:]:
            #print(s)
            #print(target[m][s])
            least_pre_list.append(target[m][s]) #把prec_list 的 score加進去
        dcg = DCG(pre_list)
        ndcg = dcg/idcg
        #print(ndcg)
        total_ndcg += ndcg
    avg_ndcg = total_ndcg/test_amount
    print('NDCG:',avg_ndcg)
    return pre_matrix,avg_ndcg

# MAP
"""
>>> y_true = np.array([0, 0, 1, 1])
>>> y_scores = np.array([0.1, 0.4, 0.35, 0.8])
>>> average_precision_score(y_true, y_scores)
"""
def MAP(target,testRS):
    print('target:',target)
    print('testRS:',testRS)
    total_prec = 0
    for u in range(test_amount):
        y_true = target[u]
        y_scores = testRS[u]
        total_prec+=average_precision_score(y_true, y_scores)
    Map_value = total_prec/test_amount
    print('MAP',Map_value)
    return Map_value

In [12]:
def testing(U, Y, A, E,Au, Ay, Aa, Av,B):
    
    test_amount = 150
    yt_test_amount = 18
    result=np.zeros((test_amount,88))
    RS=np.zeros((test_amount,88))
    #test_idx --> Test 的 index
    sum_alpha = 0
    for s in range(test_amount):
        #print(s,test_idx[s])

        yes=[]
        #sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
        sample = [i for i in range(88)]
        #sample = train_t[test_idx[s]]
        
        #sample=result_yes_id[now]
        alpha=np.zeros([len(sample)])

        for a in range(len(sample)):
            r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
            alpha_a = np.dot(Au[test_idx[s]][sample[a]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[test_idx[s]][sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]][sample[a]],
                    np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]][sample[a]],np.dot(E,np.expand_dims(all_3374[sample[a]],0).T))
            """
            relu part ...
            """
            alpha[a]=np.sum((relu(alpha_a)))*r
            """
            tanh part ...
            """
            #alpha[a]=np.sum((np.tanh(alpha_a)))*r
            
            
        mul=np.zeros((1,64))
        #print('alpha--------',alpha)
        #print('add alpha------------',np.add(alpha,0.000000001))
        added_alpha = np.add(np.maximum(alpha, 0),0.0000000001)
        norm_alpha = added_alpha/np.sum(added_alpha)
        
        #print(norm_alpha)
        #print('alpha-----------',alpha)
        #print('norm alpha--------------',norm_alpha)
        sum_alpha += np.sum(alpha)
        for i in range(len(sample)):
            mul+=norm_alpha[i]*A[sample[i]] #attention alpha*Ai part
        new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    
        
        
        for k in range(88):
            result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
            RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]],np.dot(E, all_3374[k].T))
            
    print('sum_alpha',sum_alpha)
    return RS

In [36]:
"""#With Embedding
import os 
#從grid_search_weight中找尋不同的file 
path = '../Data/grid_search_weight/Our/'
all_files = os.listdir(path)
new_all_files = []
for file in all_files:
    if 'ALL' in file:
        new_all_files.append(file)
all_files = new_all_files
print(all_files)
all_files = ['Img0.5_ALL_E150.npz','Img0.5_ALL_E160.npz','Img0.5_ALL_E170.npz']"""
files = ['ALL5.npz']
all_files = files
all_files

['ALL5.npz']

In [37]:
path = '../Data/grid_search_weight/Our/'
#all_files = ['Img0.5_ALL_E150.npz']

In [38]:
#par_files = list(set([file.split('_E')[0] for file in all_files]))
par_files = ['ALL']
print(par_files)
for par_file in par_files:
    NDCG_List = []
    MAP_List = []
    DIM_List = []
    TOP1_Prec = []
    TOP3_Prec = []
    TOP5_Prec = []
    TOP1_F1 = []
    TOP3_F1 = []
    TOP5_F1 = []
    TOP1_Recall = []
    TOP3_Recall = []
    TOP5_Recall = []
    
    csv_path = '../Data/grid_search_weight/Our/'
    for file in all_files:
        if par_file in file:
            print(file)
            par_data = np.load(path+file)
            U = par_data['U']
            Y = par_data['Y']
            A = par_data['A']
            E = par_data['E']
            #W1 = par_data['W1']
            Wu = par_data['Wu']
            Wy = par_data['Wy']
            Wa = par_data['Wa']
            Wv = par_data['Wv']
            B = par_data['B']
            """print(U.shape)
            print(Y.shape)
            print(A.shape)
            print(E.shape)
            print(B.shape)"""
            RS = testing(U, Y, A, E,Wu, Wy, Wa, Wv,B)
            target,testRS,sumtarget = getScoreMatrix(RS)
            prec_1,recall_1,f1_1 = getTOP1(target,testRS,sumtarget)
            prec_3,recall_3,f1_3 = getTOP3(target,testRS,sumtarget)
            prec_5,recall_5,f1_5 = getTOP5(target,testRS,sumtarget)
            pre_matrix,avg_ndcg = NDCG(target,testRS)
            Map_value = MAP(target,testRS)
            NDCG_List.append(avg_ndcg)
            MAP_List.append(Map_value)
            DIM_List.append(file)
            TOP1_Prec.append(prec_1)
            TOP1_Recall.append(recall_1)
            TOP1_F1.append(f1_1)
            TOP3_Prec.append(prec_3)
            TOP3_Recall.append(recall_3)
            TOP3_F1.append(f1_3)
            TOP5_Prec.append(prec_5)
            TOP5_Recall.append(recall_5)
            TOP5_F1.append(f1_5)
            print('--------------------------------------------------------------------------------------------')
    #print(NDCG_List)
    #print(DIM_List)
    result_dict = {'Dims':DIM_List,'NDCG':NDCG_List,'MAP':MAP_List,
                   'TOP1 Precision':TOP1_Prec,'TOP1 Recall':TOP1_Recall,'TOP1 F1':TOP1_F1,
                  'TOP3 Precision':TOP3_Prec,'TOP3 Recall':TOP3_Recall,'TOP3 F1':TOP3_F1,
                  'TOP5 Precision':TOP5_Prec,'TOP5 Recall':TOP5_Recall,'TOP5 F1':TOP5_F1}
    df = pd.DataFrame(result_dict)
    df.to_csv(csv_path+par_file+'.csv',index=False)

['ALL']
ALL5.npz
sum_alpha 2.9489840346822778
RS shape (150, 88)
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.8533333333333334 recall  0.15609756097560976
F1_score: 0.2639175257731959
top3
prec  0.7533333333333333 recall  0.41341463414634144
F1_score: 0.5338582677165353
top5
prec  0.6853333333333333 recall  0.6268292682926829
F1_score: 0.6547770700636942
NDCG: 0.8652579468799466
target: [[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]]
testRS: [[  3.02030123  -1.01979985   0.95287444 ...   1.17643283 -11.95212281
   -9.90817714]
 [  2.47776733  -1.25722218   2.8429347  ...  -0.09373315  -2.23774073
   -3.72977524]
 [ -6.25081016  -0.42638052  -0.70702485 ...  -9.8286897   -6.79765448
   -9.87507106]
 ...
 [ -3.42275241  -2.11855332  -4.43099991 ...  -3.94354252 -12.62768986
   -9.15580671]
 [ -0.04858852  -2.52945011  -0.15355501 ...  -4.8963626   -9

# Separate Feature Observation

In [32]:
#With Embedding
import os 
#從grid_search_weight中找尋不同的file 
path = '../Data/npy/mask_feature/result300/'
all_files = os.listdir(path)
print(all_files)

['Img0.5_no_300following_E200.npz', 'Img0.5_no_300image_E200.npz', 'Img0.5_no_300social_E200.npz', 'Img0.5_no_300text_E200.npz', 'Img0.5_no_300video_E200.npz', 'Img0.5_only_Image_E200.npz', 'Img0.5_only_text_E200.npz']


In [ ]:
par_files = list(set([file.split('_')[1]+'_'+file.split('_')[2] for file in all_files]))
print(par_files)
for par_file in par_files:
    NDCG_List = []
    MAP_List = []
    DIM_List = []
    TOP1_Prec = []
    TOP3_Prec = []
    TOP5_Prec = []
    TOP1_F1 = []
    TOP3_F1 = []
    TOP5_F1 = []
    TOP1_Recall = []
    TOP3_Recall = []
    TOP5_Recall = []
    
    #csv_path = '../Data/grid_search_weight/CSV_Tan/'
    csv_path = '../Data/npy/mask_feature/csv/'
    for file in all_files:
        if par_file in file:
            print(file)
            par_data = np.load(path+file)
            U = par_data['U']
            Y = par_data['Y']
            A = par_data['A']
            E = par_data['E']
            #W1 = par_data['W1']
            Wu = par_data['Wu']
            Wy = par_data['Wy']
            Wa = par_data['Wa']
            Wv = par_data['Wv']
            B = par_data['B']
            RS = testing(U, Y, A, E,Wu, Wy, Wa, Wv,B)
            target,testRS,sumtarget = getScoreMatrix(RS)
            print('target',target)
            prec_1,recall_1,f1_1 = getTOP1(target,testRS,sumtarget)
            prec_3,recall_3,f1_3 = getTOP3(target,testRS,sumtarget)
            prec_5,recall_5,f1_5 = getTOP5(target,testRS,sumtarget)
            pre_matrix,avg_ndcg = NDCG(target,testRS)
            Map_value = MAP(target,pre_matrix)
            NDCG_List.append(avg_ndcg)
            MAP_List.append(Map_value)
            DIM_List.append(file.split('_')[1])
            TOP1_Prec.append(prec_1)
            TOP1_Recall.append(recall_1)
            TOP1_F1.append(f1_1)
            TOP3_Prec.append(prec_3)
            TOP3_Recall.append(recall_3)
            TOP3_F1.append(f1_3)
            TOP5_Prec.append(prec_5)
            TOP5_Recall.append(recall_5)
            TOP5_F1.append(f1_5)
            print('--------------------------------------------------------------------------------------------')
    #print(NDCG_List)
    #print(DIM_List)
    result_dict = {'Dims':DIM_List,'NDCG':NDCG_List,'MAP':MAP_List,
                   'TOP1 Precision':TOP1_Prec,'TOP1 Recall':TOP1_Recall,'TOP1 F1':TOP1_F1,
                  'TOP3 Precision':TOP3_Prec,'TOP3 Recall':TOP3_Recall,'TOP3 F1':TOP3_F1,
                  'TOP5 Precision':TOP5_Prec,'TOP5 Recall':TOP5_Recall,'TOP5 F1':TOP5_F1}
    df = pd.DataFrame(result_dict)
    df.to_csv(csv_path+par_file+'.csv',index=False)

In [20]:
#For VBPR
par_files = list(set([file.split('_')[1] for file in all_files]))
print(par_files)
for par_file in par_files:
    NDCG_List = []
    MAP_List = []
    DIM_List = []
    TOP1_Prec = []
    TOP3_Prec = []
    TOP5_Prec = []
    TOP1_F1 = []
    TOP3_F1 = []
    TOP5_F1 = []
    TOP1_Recall = []
    TOP3_Recall = []
    TOP5_Recall = []
    
    csv_path = '../Data/grid_search_weight/CSV_Tan/'
    #csv_path = '../Data/npy/mask_feature/csv/'
    for file in all_files:
        if par_file in file:
            print(file)
            par_data = np.load(path+file)
            U = par_data['U']
            Y = par_data['Y']
            A = par_data['A']
            E = par_data['E']
            #W1 = par_data['W1']
            Wu = parRS,sumtarget = getScoreMatrix(RS)
            print('target',target)
            prec_1,recall_1,f1_1 = getTOP1(target,testRS,sumtarget)
            prec_3,recall_3,f1_3 = getTOP3(target,testRS,sumtarget)
            prec_5,recall_5,f1_5 = getTOP5(target,testRS,sumtarget)
            pre_matrix,avg_data['Wu']
            Wy = par_data['Wy']
            Wa = par_data['Wa']
            Wv = par_data['Wv']
            B = par_data['B']
            RS = testing(U, Y, A, E,Wu, Wy, Wa, Wv,B)
            target,test_ndcg = NDCG(target,testRS)
            Map_value = MAP(target,pre_matrix)
            NDCG_List.append(avg_ndcg)
            MAP_List.append(Map_value)
            DIM_List.append(file.split('_')[1])
            TOP1_Prec.append(prec_1)
            TOP1_Recall.append(recall_1)
            TOP1_F1.append(f1_1)
            TOP3_Prec.append(prec_3)
            TOP3_Recall.append(recall_3)
            TOP3_F1.append(f1_3)
            TOP5_Prec.append(prec_5)
            TOP5_Recall.append(recall_5)
            TOP5_F1.append(f1_5)
            print('--------------------------------------------------------------------------------------------')
    #print(NDCG_List)
    #print(DIM_List)
    result_dict = {'Dims':DIM_List,'NDCG':NDCG_List,'MAP':MAP_List,
                   'TOP1 Precision':TOP1_Prec,'TOP1 Recall':TOP1_Recall,'TOP1 F1':TOP1_F1,
                  'TOP3 Precision':TOP3_Prec,'TOP3 Recall':TOP3_Recall,'TOP3 F1':TOP3_F1,
                  'TOP5 Precision':TOP5_Prec,'TOP5 Recall':TOP5_Recall,'TOP5 F1':TOP5_F1}
    df = pd.DataFrame(result_dict)
    df.to_csv(csv_path+par_file+'.csv',index=False)

['onlyImg.npz', 'ImgVideo.npz']
vbpr_onlyImg.npz


KeyError: 'A is not a file in the archive'

In [ ]:
par_files = list(set([file.split('_Edims')[0] for file in all_files]))
print(par_files)
for par_file in par_files:
    NDCG_List = []
    MAP_List = []
    DIM_List = []
    TOP1_Prec = []
    TOP3_Prec = []
    TOP5_Prec = []
    TOP1_F1 = []
    TOP3_F1 = []
    TOP5_F1 = []
    TOP1_Recall = []
    TOP3_Recall = []
    TOP5_Recall = []
    
    csv_path = '../Data/grid_search_weight/CSV_Tan/'
    for file in all_files:
        if par_file in file:
            print(file)
            par_data = np.load(path+file)
            U = par_data['U']
            Y = par_data['Y']
            A = par_data['A']
            E = par_data['E']
            #W1 = par_data['W1']
            Wu = par_data['Wu']
            Wy = par_data['Wy']
            Wa = par_data['Wa']
            Wv = par_data['Wv']
            B = par_data['B']
            RS = testing(U, Y, A, E,Wu, Wy, Wa, Wv,B)
            target,testRS,sumtarget = getScoreMatrix(RS)
            prec_1,recall_1,f1_1 = getTOP1(target,testRS,sumtarget)
            prec_3,recall_3,f1_3 = getTOP3(target,testRS,sumtarget)
            prec_5,recall_5,f1_5 = getTOP5(target,testRS,sumtarget)
            pre_matrix,avg_ndcg = NDCG(target,testRS)
            Map_value = MAP(target,pre_matrix)
            NDCG_List.append(avg_ndcg)
            MAP_List.append(Map_value)
            DIM_List.append(file.split('Edims')[1].split('.npz')[0])
            TOP1_Prec.append(prec_1)
            TOP1_Recall.append(recall_1)
            TOP1_F1.append(f1_1)
            TOP3_Prec.append(prec_3)
            TOP3_Recall.append(recall_3)
            TOP3_F1.append(f1_3)
            TOP5_Prec.append(prec_5)
            TOP5_Recall.append(recall_5)
            TOP5_F1.append(f1_5)
            print('--------------------------------------------------------------------------------------------')
    #print(NDCG_List)
    #print(DIM_List)
    result_dict = {'Dims':DIM_List,'NDCG':NDCG_List,'MAP':MAP_List,
                   'TOP1 Precision':TOP1_Prec,'TOP1 Recall':TOP1_Recall,'TOP1 F1':TOP1_F1,
                  'TOP3 Precision':TOP3_Prec,'TOP3 Recall':TOP3_Recall,'TOP3 F1':TOP3_F1,
                  'TOP5 Precision':TOP5_Prec,'TOP5 Recall':TOP5_Recall,'TOP5 F1':TOP5_F1}
    df = pd.DataFrame(result_dict)
    df.to_csv(csv_path+par_file+'.csv',index=False)

# NDCG For Different Dimensions

In [111]:
######## code here !!!!

(10, 20)

In [69]:
def topN(sortlist,n):
    topList = []
    for i in range(n):
        topList.append(sortlist.index(max(sortlist)))
        #print(max(sortlist))
        #print(sortlist.index(max(sortlist)))
        sortlist[sortlist.index(max(sortlist))] = -1000000000
    return topList

In [70]:
count_0_all = []
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),int(np.sum(target[i])))
    count_0_all.append(top_0)
    print(top_0)

acc_0 = 0
total = 0
for i in range(len(count_0_all)):
    for j in range(len(count_0_all[i])):
        #print(int(np.sum(target[i])))
        total+=int(np.sum(target[i]))
        if count_0_all[i][j] < int(np.sum(target[i])): #代表是0或1 (也就是target)
            acc_0 += 1
avg_acc = acc_0/100
print('avg_accuarcy for count_0:',avg_acc)
print(acc_0)

[0, 2, 1]
[3, 6, 2]
[1, 2, 13, 4, 5, 3, 17]
[1, 10, 8, 2, 5, 4]
[4, 1, 12]
[1, 15, 11]
[2, 1, 10, 0]
[3, 2, 5, 7, 4, 18]
[3, 18, 8, 5, 13]
[12, 16, 7, 0, 9]
avg_accuarcy for count_0: 0.24
24


# TOP1,TOP3,TOP5

In [71]:
def F1_score(prec,rec):
    f1 = (2*prec*rec)/(prec+rec)
    return f1

# NDCG 

In [172]:
a_list = [0.02162337, 0.02162659, 0.00240219, 0.00780985, 0.02162318, 0.00780592, 0.00780849]
softmax(a_list)

array([0.14409628, 0.14409674, 0.14135303, 0.14211949, 0.14409625,
       0.14211893, 0.14211929])

In [7]:

user = []
for i in range(3):
    temp_t = []
    temp_t.append(1)
    temp_t.append(2)
    temp_t.append(3)
    user.append(temp_t)
np.array(user)

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3]])

In [40]:
y_true = np.array([0, 0, 1, 1])
y_scores = np.array([-1, 1, 0,2])
average_precision_score(y_true, y_scores)

0.8333333333333333